# All imports

In [1]:
#I'm using BeautifulSoup for scraping
import requests
from bs4 import BeautifulSoup
import sqlite3
import os.path


In [2]:
#Url for CNN Money’s Market Movers website

url = "https://money.cnn.com/data/hotstocks/"

# Get the HTML of URL

In [3]:
#Step 1: Get the HTML of the above URL using requests.get(url) method.
#If print statement is uncommented and run you can see html of website

r = requests.get(url)
htmlContent = r.content

#print(htmlContent)

# Parse HTML

In [4]:
#Step 2: Parse the HTML using html parser

soup = BeautifulSoup(htmlContent, 'html.parser')

# print(soup.prettify)

# Extract Tickers

In [5]:
#Step 3: HTML Tree Traversal
#Since all of the tickers are contained inside of a table's anchor tag, I'm extracting tickers and appending them in a list
#This is all we need to extract from CNN's website

tbody = soup.find('table')
anchor = tbody.find_all('a')
tickers = []
for a in anchor:
    tickers.append(a.text)

print(tickers)

['F', 'GE', 'BAC', 'XOM', 'FCX', 'PFE', 'CCL', 'T', 'WFC', 'BSX']


In [6]:
#URL Concatenation for next website to be scraped
#We need to change the URL to open the website for each ticker in tickers

first_part = "https://finance.yahoo.com/quote/";
third_part = "?p=";
fifth_part = "&.tsrc=fin-srch-v1";

# Get data on each ticker from Yahoo Finance and file writing

In [7]:
#stocksdb will append all information about each stock in the order with which it will be stored in database. 
stockdb = []

#records will append all information about each stock in the order with which it will be stored in stocks.txt.
records = []

#Loop for each ticker in tickers

for ticker in tickers:
    try:
        #append yahoo finance url for each ticker
        yahoo_url = first_part + ticker + third_part + ticker + fifth_part;
        r2 = requests.get(yahoo_url)
        htmlContent2 = r2.content
        soup2 = BeautifulSoup(htmlContent2, 'html.parser')

        #Open price
        all_open_price = soup2.select("span[data-reactid*='103']")[0].text

        #Avg_volume and avg_volume value with comma removed so it can be stored as integer type
        all_avg_volume = soup2.find('span', string = 'Avg. Volume').find_next().find_next().text
        avg_volume_without_commas = all_avg_volume.replace(',', '')

        #PE_ratio value is 0 where N/A
        all_PE_ratio = soup2.find('span', string = 'PE Ratio (TTM)').find_next().find_next().text
        if(all_PE_ratio == "N/A"):
            all_PE_ratio = "0";

        #appending it in the manner I want for the stocks.txt file (commas and new line)
        stock_info = ticker + "," + all_open_price + "," + avg_volume_without_commas + "," + all_PE_ratio +','+'\n'
        records.append(stock_info)

        #appending it in the manner I want for the database
        stock = (ticker,all_open_price,avg_volume_without_commas,all_PE_ratio)
        stockdb.append(stock)
    except(AttributeError, KeyError) as er:
        pass
        
            
            
            
#opening and writing to a file (a new file is made oneach execution)
file = open('stocks.txt','w')
line = "".join(records)
file.write(line)
file.close()


# Check if DB already exists

In [8]:
#if a DB already exists it will delete it and make a new one. Each execution of the program makes a new database.
try:
    checkDB = os.path.isfile('./StocksDatabase.db')
    if(checkDB==True):
        os.remove('StocksDatabase.db')
except:
    print("error: database already exists, go ahead")


# Connecting and creating database table

In [9]:
#Connecting to a database
conn = sqlite3.connect('StocksDatabase.db')

In [10]:
c = conn.cursor()

In [11]:
# Create table
try:
    c.execute('''CREATE TABLE StocksTable
             (Ticker text, OpenPrice real, AvgVolume integer, PERatio real)''')
except: 
    print("error: table already exists, go ahead")

In [12]:
c.executemany('INSERT INTO StocksTable VALUES (?,?,?,?)', stockdb)

In [13]:
conn.commit()
c.close()
#Checked with SQlite, all data present